In [1]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
from implicit.nearest_neighbours import TFIDFRecommender
import scipy.sparse as sp
import warnings
warnings.simplefilter("ignore")
from pprint import pprint
from tqdm._tqdm_notebook import tqdm_notebook

tqdm_notebook.pandas()

In [2]:
DATA_PATH = 'data'

In [3]:
with open(os.path.join(DATA_PATH, 'catalogue.json'), 'r') as f:
    catalogue = json.load(f)
    
catalogue = {int(k): v for k, v in catalogue.items()}

In [ ]:
len(catalogue)

In [ ]:
pprint(catalogue[100])

 - `attributes` — мешок атрибутов
 - `availability` — доступность (может содержать значения `purchase`, `rent` и `subscription`)
 - `duration` — длительность в минутах, округлённая до десятков (продолжительность серии для сериалов и многосерийных фильмов)
 - `feature_1..5` — пять анонимизированных вещественных и порядковых признаков
 - `type` — принимает значения `movie`, `multipart_movie` или `series`

---

`test_users.json` содержит список пользователей, для которых необходимо построить предсказание

In [ ]:
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])

---

`transactions.csv` — список всех транзакций за определённый период времени

In [4]:
transactions = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    }
)

In [ ]:
transactions.head(3)

 - `element_uid` — идентификатор элемента
 - `user_uid` — идентификатор пользователя
 - `consumption_mode` — тип потребления (`P` — покупка, `R` — аренда, `S` — просмотр по подписке)
 - `ts` — время совершения транзакции или начала просмотра в случае просмотра по подписке
 - `watched_time` — число просмотренных по транзакции секунд
 - `device_type` — анонимизированный тип устройства, с которого была совершена транзакция или начат просмотр
 - `device_manufacturer` — анонимизированный производитель устройства, с которого была совершена транзакция или начат просмотр

---

`ratings.csv` содержит информацию о поставленных пользователями оценках

In [5]:
ratings = pd.read_csv(
    os.path.join(DATA_PATH, 'ratings.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64,
        'rating': np.uint8
    }
)

 - `rating` — поставленный пользователем рейтинг (от `0` до `10`)

In [ ]:
ratings.head(3)

---

`bookmarks.csv` содержит информацию об элементах, добавленных пользователями в список «Избранное»

In [6]:
bookmarks = pd.read_csv(
    os.path.join(DATA_PATH, 'bookmarks.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64
    }
)

In [ ]:
bookmarks.head(3)

## Решение

Для начала построим список элементов, которые тестовые пользователи уже купили или посмотрели по подписке: они не смогут купить их второй раз, а просмотр по подписке второй раз маловероятен, поэтому мы захотим отфильтровать такие элементы из финального ответа.

Точно так же можно поступить и с рейтингами и добавлениями в избранное, если это будет казаться правильным.

In [ ]:
from collections import defaultdict

filtered_elements = defaultdict(set)

for user_uid, element_uid in tqdm_notebook(transactions.loc[:, ['user_uid', 'element_uid']].values):
    if user_uid not in test_users:
        continue
    filtered_elements[user_uid].add(element_uid)

---

Для примера мы воспользуемся методом K ближайших соседей, реализованным в библиотеке `implicit`. В качестве данных используем только информацию о рейтингах.

Необходимо построить разреженную матрицу, где строкам будут соответствовать элементы, столбцам — пользователи, а на пересечении пользователя и элемента будет находиться количественная оценка степени их взаимодействия, если таковое имело место.

Не забудем добавить `1` к рейтингу, чтобы избежать деления на ноль во время вычисления `tf-idf`.

In [ ]:
ratings['user_uid'] = ratings['user_uid'].astype('category')
ratings['element_uid'] = ratings['element_uid'].astype('category')

ratings_matrix = sp.coo_matrix(
    (ratings['rating'].astype(np.float32) + 1,
        (
            ratings['element_uid'].cat.codes.copy(),
            ratings['user_uid'].cat.codes.copy()
        )
    )
)

ratings_matrix = ratings_matrix.tocsr()

In [ ]:
ratings_matrix

In [ ]:
sparsity = ratings_matrix.nnz / (ratings_matrix.shape[0] * ratings_matrix.shape[1])
print('Sparsity: %.6f' % sparsity)

Обучить модель крайне просто.

In [ ]:
from implicit.nearest_neighbours import TFIDFRecommender

model = TFIDFRecommender()
model.fit(ratings_matrix)

---

In [ ]:
ratings_matrix_T = ratings_matrix.T.tocsr()

Отображения из оригинальной категории во внутреннюю пригодится нам в дальнейшем.

In [ ]:
user_uid_to_cat = dict(zip(
    ratings['user_uid'].cat.categories,
    range(len(ratings['user_uid'].cat.categories))
))

In [ ]:
element_uid_to_cat = dict(zip(
    ratings['element_uid'].cat.categories,
    range(len(ratings['element_uid'].cat.categories))
))

In [ ]:
filtered_elements_cat = {k: [element_uid_to_cat.get(x, None) for x in v] for k, v in filtered_elements.items()}

---

В метод `model.recommend` мы передаём идентификатор пользователя, который получаем обратным преобразованием из категории, транспонированную матрицу взаимодействий, число необходимых рекомендаций и список элементов, которые мы договорились фильтровать из ответа.

Возвращает метод список пар (`element_cat`, `score`), отсортированный по вторым элементам. Из него необходимо достать все первые элементы пар и из категории преобразовать их к `element_uid`.

**Важно:** Не все тестовые пользователи есть в `ratings.csv` и не все из них есть в `transactions.csv`. Используя только один источник данных мы не можем построить полное предсказание. Такой ответ с неполным числом пользователей бдет принят системой, но при вычислении средней метрики метрика для отсутствующих пользователей будет принята равной нулю.

In [ ]:
result = {}

for user_uid in tqdm.tqdm(test_users):
    # transform user_uid to model's internal user category
    try:
        user_cat = user_uid_to_cat[user_uid]
    except LookupError:
        continue
    
    # perform inference
    recs = model.recommend(
        user_cat,
        ratings_matrix_T,
        N=20,
        filter_already_liked_items=True,
        filter_items=filtered_elements_cat.get(user_uid, set())
    )
    
    # drop scores and transform model's internal elelemnt category to element_uid for every prediction
    # also convert np.uint64 to int so it could be json serialized later
    result[user_uid] = [int(ratings['element_uid'].cat.categories[i]) for i, _ in recs]

In [ ]:
len(result)

Используя только информацию о рейтингах мы смогли построить предсказание для `13251` из `50000` тестовых пользователей. Ровно в таком виде ответы и стоит сохранить для отправки.

In [ ]:
with open('answer.json', 'w') as f:
    json.dump(result, f)

## Отсюда мое

In [ ]:
ratings = pd.read_csv(
    os.path.join(DATA_PATH, 'ratings.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64,
        'rating': np.uint8
    }
)

In [19]:
def _cv_split(n_splits, data_train): # old version
    """
    Разделяем данные на трейн юзеров и тест юзеров по уникальным пользователям
    :n_splits: кол-во фолдов
    :data_train: данные для обучения
    """
    unique_users = np.unique(data_train['user_uid'])
    split_data = np.array_split(unique_users, n_splits)
    
    for fold in range(n_splits):
        train_users, test_users = np.hstack([x for i,x in enumerate(split_data) if i != fold]), split_data[fold]
        yield train_users, test_users

In [20]:
def cv_split(n_splits, data_train): # new version
    """
    Разделяем данные на трейн юзеров и тест юзеров по уникальным пользователям
    :n_splits: кол-во фолдов
    :data_train: данные для обучения
    """
    unique_users = np.unique(data_train['user_uid'])
    split_data = np.array_split(unique_users, n_splits)
    
    for fold in range(n_splits):
        test_users = split_data[fold]
        train_users = np.setdiff1d(unique_users, test_users)
        yield train_users, test_users

In [8]:
def _get_dfs(data_train, count, *cv_fold, data=transactions): # old version
    """
    Оставляем в тесте и трейне только тех у кого более или равно count кол-во фильмов,
    выдаем данные для обучения и оценки
    :data_train: данные для обучения
    :count: кол-во фильмов, для трейна и предикта, также это кол-во фильмов для предсказаний
    :data: данные для выбора только тех юзеров у которых есть определенное кол-во фильмов
    true_data - словарь с релевантным 20 (count) кол-вом фильмов, для теста, с которым будет сверятся на метрике
    """
    data = data[data.groupby('user_uid')['element_uid'].transform('size') >= count]
    unique_users = np.unique(data['user_uid'])
    
    data.sort_values(by=['user_uid', 'watched_time'], ascending=False, inplace=True)
    data = data.groupby('user_uid').head(count) ## ВОТ ЭТО ДЕЙСТВИЕ НАДО ДЕЛАТЬ ИЛИ НЕТ?
    data = data.groupby('user_uid').element_uid.apply(list).to_dict()
      
    train_users, test_users = cv_fold[0], cv_fold[1]
    train_users = np.intersect1d(train_users, unique_users)
    test_users = np.intersect1d(test_users, unique_users)
    train_df = data_train[data_train['user_uid'].isin(unique_users)]
    test_df = data_train[data_train['user_uid'].isin(unique_users)]
    
    true_data = {k:v for k,v in data.items() if k in test_users}
    
    return train_df, test_df, true_data

In [9]:
def get_dfs(data_train, count, *cv_fold, data=transactions): # new version
    """
    Оставляем в тесте и трейне только тех у кого более или равно count кол-во фильмов,
    выдаем данные для обучения и оценки
    :data_train: данные для обучения
    :count: кол-во фильмов, для трейна и предикта, также это кол-во фильмов для предсказаний
    :data: данные для выбора только тех юзеров у которых есть определенное кол-во фильмов
    true_data - словарь с релевантным 20 (count) кол-вом фильмов, для теста, с которым будет сверятся на метрике
    """
    unique_users = np.unique(data['user_uid'])
    
    data.sort_values(by=['user_uid', 'watched_time'], ascending=False, inplace=True)
    data = data.groupby('user_uid').element_uid.apply(list).to_dict()
      
    train_users, test_users = cv_fold[0], cv_fold[1]

    train_df = data_train[data_train['user_uid'].isin(unique_users)]
    test_df = data_train[data_train['user_uid'].isin(unique_users)]
    
    true_data = {k:v for k,v in data.items() if k in test_users}
    
    return train_df, test_df, true_data

In [10]:
def sparse_df(train_df, test_df):   
    """
    Соединяем train_df и test_df воедино (вот этот момент мне не ясен, но вроде так надо). 
    Получаем разреженную матрицу от всего общего df, траспонированную, user_uid_to_cat, df
    """
    df = train_df.append(test_df).sort_index()
    df['user_uid'] = df['user_uid'].astype('category')
    df['element_uid'] = df['element_uid'].astype('category')

    ratings_matrix = sp.coo_matrix(
        (df['rating'].astype(np.float32) + 1,
            (
                df['element_uid'].cat.codes.copy(),
                df['user_uid'].cat.codes.copy()
            )
        )
    )

    user_uid_to_cat = dict(zip(
        df['user_uid'].cat.categories,
        range(len(df['user_uid'].cat.categories))
    ))

    ratings_matrix = ratings_matrix.tocsr()
    ratings_matrix_T = ratings_matrix.T.tocsr()
    
    return ratings_matrix, ratings_matrix_T, user_uid_to_cat, df

In [11]:
def model_fit(ratings_matrix):
    """ 
    Обучение модели на разреженной матрице
    """
    model = TFIDFRecommender()
    model.fit(ratings_matrix)
    
    return model

In [12]:
def _get_predicted_data(test_df, user_uid_to_cat, ratings_matrix_T, df, count): # old version
    """
    Получаем предсказзанный словарь с count - необходимым кол-вом фильмов
    """
    
    predicted_data = {}

    for user_uid in tqdm.tqdm(test_df.user_uid.values):
        try:
            user_cat = user_uid_to_cat[user_uid]
        except LookupError:
            continue

        recs = model.recommend(
            user_cat,
            ratings_matrix_T,
            N=count,)
    #         filter_already_liked_items=True,
    #         filter_items=filtered_elements_cat.get(user_uid, set())
    #     )

        predicted_data[user_uid] = [int(df['element_uid'].cat.categories[i]) for i, _ in recs]
        
    return predicted_data

In [13]:
def get_predicted_data(test_df, user_uid_to_cat, ratings_matrix_T, df, count): # new version
    """
    Получаем предсказанный словарь с count - необходимым кол-вом фильмов
    """
    
    predicted_data = {}
    movies_top = transactions['element_uid'].value_counts().head(count).index.tolist()
    
    for user_uid in tqdm.tqdm(test_df.user_uid.values):
        try:
            user_cat = user_uid_to_cat[user_uid]
            
            recs = model.recommend(
                user_cat,
                ratings_matrix_T,
                N=count)
            
            predicted_data[user_uid] = [int(df['element_uid'].cat.categories[i]) for i, _ in recs]    
            
        except LookupError:
            predicted_data[user_uid] = movies_top
        
    return predicted_data

In [14]:
# Скопировал код с https://habr.com/ru/company/okko/blog/439180/ хз правильно ли отредактировал с CPython
def average_precision(data_true, data_predicted, k) -> float:

    if not data_true:
        raise ValueError('data_true is empty')

    average_precision_sum = 0.0

    for key, items_true in data_true.items():
        items_predicted = data_predicted.get(key, [])

        n_items_true = len(items_true)
        n_items_predicted = min(len(items_predicted), k)

        if n_items_true == 0 or n_items_predicted == 0:
            continue

        n_correct_items = 0
        precision = 0.0

        for item_idx in range(n_items_predicted):
            if items_predicted[item_idx] in items_true:
                n_correct_items += 1
                precision += n_correct_items / (item_idx + 1)

        average_precision_sum += precision / min(n_items_true, k)

    return average_precision_sum / len(data_true)

In [15]:
def metric(true_data, predicted_data, k):
    true_data_set = {k: set(v) for k, v in true_data.items()}

    return average_precision(true_data_set, predicted_data, k=k)

### По новым методам

In [16]:
n_splits = 5
k_list = [10, 20]
count = 20
metrics = {}
for k in k_list:
    counter = cv_split(n_splits, ratings)
    temp_metrics = np.zeros(n_splits)
    i = 0
    for it in counter:
        train_df, test_df, true_data = get_dfs(ratings, count, *it)
        ratings_matrix, ratings_matrix_T, user_uid_to_cat, df = sparse_df(train_df, test_df)
        model = model_fit(ratings_matrix)
        predicted_data = get_predicted_data(test_df, user_uid_to_cat, ratings_matrix_T, df, count)
        temp_metrics[i] = metric(true_data, predicted_data, k)
        i += 1
    metrics[k] = temp_metrics

100%|██████████| 435189/435189 [01:13<00:00, 5887.90it/s]


In [17]:
metrics

{10: array([0.05862126, 0.05793591, 0.0570175 , 0.05768103, 0.0579142 ]),
 20: array([0.04925117, 0.04822313, 0.04775214, 0.04845391, 0.04833178])}

### По старым методам

In [21]:
n_splits = 5
k_list = [10, 20]
count = 20
metrics = {}
for k in k_list:
    counter = _cv_split(n_splits, ratings)
    temp_metrics = np.zeros(n_splits)
    i = 0
    for it in counter:
        train_df, test_df, true_data = _get_dfs(ratings, count, *it)
        ratings_matrix, ratings_matrix_T, user_uid_to_cat, df = sparse_df(train_df, test_df)
        model = model_fit(ratings_matrix)
        predicted_data = _get_predicted_data(test_df, user_uid_to_cat, ratings_matrix_T, df, count)
        temp_metrics[i] = metric(true_data, predicted_data, k)
        i += 1
    metrics[k] = temp_metrics

100%|██████████| 305839/305839 [00:55<00:00, 5557.48it/s]


In [22]:
metrics

{10: array([0.0458047 , 0.04547763, 0.04599957, 0.04604166, 0.04529647]),
 20: array([0.0287629 , 0.02851868, 0.02888257, 0.02894626, 0.02833982])}